<a href="https://colab.research.google.com/github/mrzaizai2k/Vietnamese_stocks_forecasting/blob/main/PyTorch_Forecasting_Multi_Target_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pytorch-forecasting

     |████████████████████████████████| 102kB 2.9MB/s 
     |████████████████████████████████| 266kB 15.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 22.2MB 6.1MB/s 
     |████████████████████████████████| 102kB 5.3MB/s 
     |████████████████████████████████| 686kB 57.0MB/s 
     |████████████████████████████████| 163kB 55.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 276kB 53.0MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 112kB 34.5MB/s 
     |████████████████████████████████| 133kB 57.7MB/s 
  Created wheel for optuna: filename=optuna-2.3.0-cp36-none-any.whl size=359761 sha256=a389a36f857655a57ea5ba86a4cb7c48cfc5ea9cbaff481379290ab9f6f50a7d
  Stored in directory: /root/.cache/pip/wheels/fa/91/19/64

In [ ]:
import warnings
import numpy as np
from pandas.core.common import SettingWithCopyWarning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data.examples import get_stallion_data
from pytorch_forecasting.data.encoders import EncoderNormalizer
from pytorch_forecasting.metrics import QuantileLoss, MultiLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
warnings.simplefilter("error", category=SettingWithCopyWarning)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
# Originally from https://github.com/jdb78/pytorch-forecasting/blob/master/tests/conftest.py#L27
def get_test_data():
    data = get_stallion_data()
    data["month"] = data.date.dt.month.astype(str)
    data["log_volume"] = np.log1p(data.volume)
    data["weight"] = 1 + np.sqrt(data.volume)

    data["time_idx"] = data["date"].dt.year * 12 + data["date"].dt.month
    data["time_idx"] -= data["time_idx"].min()

    special_days = [
        "easter_day",
        "good_friday",
        "new_year",
        "christmas",
        "labor_day",
        "independence_day",
        "revolution_day_memorial",
        "regional_games",
        "fifa_u_17_world_cup",
        "football_gold_cup",
        "beer_capital",
        "music_fest",
    ]
    data[special_days] = data[special_days].apply(lambda x: x.map({0: "", 1: x.name})).astype("category")

    data = data[lambda x: x.time_idx < 10]  # downsample
    data = data.assign(volume1=lambda x: x.volume) # add second target
    return data

test_data = get_test_data()

In [ ]:
# Originally from https://github.com/jdb78/pytorch-forecasting/blob/master/tests/test_data.py#L476
training = TimeSeriesDataSet(
        test_data,
        time_idx="time_idx",
        target=["volume", "volume1"],
        group_ids=["agency", "sku"],
        max_encoder_length=5,
        max_prediction_length=2,
        min_prediction_length=1,
        min_encoder_length=1,
        time_varying_known_reals=["price_regular"],
        scalers={"price_regular": EncoderNormalizer()}#,
        #**kwargs,
)

In [ ]:
# Originally from https://pytorch-forecasting.readthedocs.io/en/latest/tutorials/stallion.html
validation = TimeSeriesDataSet.from_dataset(training, test_data, predict=True, stop_randomization=True)
batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

AssertionError: ignored

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

trainer = pl.Trainer(
    max_epochs=30,
    gpus=0,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  # coment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=[7,7],  # 7 quantiles by default, two outputs for Multi-Target
    loss=MultiLoss([QuantileLoss(),QuantileLoss()]), # using MultiLoss for Multi-Target
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
# remove logging and artificial epoch size
tft.hparams.log_interval = -1
tft.hparams.log_val_interval = -1
trainer.limit_train_batches = 1.0
# run learning rate finder
res = trainer.tuner.lr_find(
    tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader, min_lr=1e-5, max_lr=1e2
)
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()
tft.hparams.learning_rate = res.suggestion()

In [ ]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

In [ ]:
# make a prediction on entire validation set
preds, index = tft.predict(val_dataloader, return_index=True, fast_dev_run=True)
preds, index

In [ ]:
# tune
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=200,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,
)